In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

In [2]:
import os
os.chdir("../../")

In [3]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma(persist_directory="chroma", embedding_function=embeddings)

# Create retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [4]:
retriever.invoke("What animated project did Adam West and Burt Ward announce at the Mad Monster Party in Charlotte, NC?")

[Document(page_content='(The Hollywood Reporter)The skies over Gotham City might become bright once again in 2016, with Adam West and Burt Ward promising a return to their career-high roles of Batman and Robin as part of an upcoming animated project next year. The comments took place at the Mad Monster Party in Charlotte, NC this weekend, with Ward somewhat uncertainly telling the crowd that they "can look forward to a new animated "Batman" full length, 90-minute feature, coming out on the 50th anniversary." (That would be the 50th anniversary of the 1960s Batman TV series, which launched Jan. 12, 1966.) Why more'),
 Document(page_content='be the 50th anniversary of the 1960s Batman TV series, which launched Jan. 12, 1966.) Why more \'Star Wars\' actors haven\'t become stars . The animated movie is described by Ward as "one of possibly two, but for sure one, that Adam and I are going to be doing the voices for," although he didn\'t say anything else about the project. In fact, as video

In [6]:
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.

Context: {context}

Question: {question}

Helpful Answer:"""

custom_template = PromptTemplate.from_template(template)

In [13]:
# Initialize language model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [14]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def ragas_output_parser(docs):
    return [doc.page_content for doc in docs]

def rag_chain_setup(retriever, llm):
    generator = custom_template | llm | StrOutputParser()

    context_retriever = RunnableParallel(
        {
            "context": retriever | format_docs,
            "question": RunnablePassthrough(),
        }
    )

    filter_langsmith_dataset = RunnableLambda(
        lambda x: x["question"] if isinstance(x, dict) else x
    )

    rag_chain = RunnableParallel(
        {
            "question": filter_langsmith_dataset,
            "answer": filter_langsmith_dataset | context_retriever | generator,
            "contexts": filter_langsmith_dataset
            | retriever
            | ragas_output_parser,
        }
    )
    return rag_chain

In [15]:
rag_chain = rag_chain_setup(retriever, llm)

In [16]:
q = "What animated project did Adam West and Burt Ward announce at the Mad Monster Party in Charlotte, NC?"
res = rag_chain.invoke(q)

In [17]:
res

{'question': 'What animated project did Adam West and Burt Ward announce at the Mad Monster Party in Charlotte, NC?',
 'answer': 'Adam West and Burt Ward announced a new animated "Batman" full-length, 90-minute feature coming out on the 50th anniversary of the 1960s Batman TV series.',
 'contexts': ['(The Hollywood Reporter)The skies over Gotham City might become bright once again in 2016, with Adam West and Burt Ward promising a return to their career-high roles of Batman and Robin as part of an upcoming animated project next year. The comments took place at the Mad Monster Party in Charlotte, NC this weekend, with Ward somewhat uncertainly telling the crowd that they "can look forward to a new animated "Batman" full length, 90-minute feature, coming out on the 50th anniversary." (That would be the 50th anniversary of the 1960s Batman TV series, which launched Jan. 12, 1966.) Why more',
  'be the 50th anniversary of the 1960s Batman TV series, which launched Jan. 12, 1966.) Why more \

In [ ]:
from ragas.integrations.langchain import EvaluatorChain
from ragas.integrations.langsmith import evaluate

# the metric for evaluation
from ragas.metrics import (
    answer_correctness,
    faithfulness,
    answer_relevancy,
    context_precision,
)
# Dataset that was uploaded in LangSmith 
dataset_name = "cnn_dailymail_testset"

# evaluate just llms
run = evaluate(
    dataset_name=dataset_name,
    llm_or_chain_factory=rag_chain,
    experiment_name="basic_rag",
    metrics=[
        answer_correctness,
        faithfulness,
        answer_relevancy,
        context_precision,
    ],
    verbose=True,
)

### Get an example from Langsmith

In [11]:
# get one example question for the dataset for testing
from langsmith import Client

client = Client()
examples = list(client.list_examples(dataset_name=dataset_name, limit=1))

q = examples[0].inputs
q


{'question': 'What animated project did Adam West and Burt Ward announce at the Mad Monster Party in Charlotte, NC?'}

### Test the rag chain with the langsmith example

In [12]:
# check with the example question to see if everything is working
get_answer = RunnableLambda(lambda x: x["answer"])
resp = (rag_chain | get_answer).invoke(q)
resp

"I don't know."